In [220]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))
import warnings
from scipy.stats import mode
import plotly.graph_objects as go
import datetime

### data access

In [179]:
next_order = pd.read_csv("frequency_prediction.csv")
weekly_income = pd.read_csv("monetary_value_for_clv_calculation.csv").rename(columns={'0': "user_id"})
last_30_orders_per_users = pd.read_csv("transactions_per_users.csv")
last_30_orders_income = pd.read_csv("transactions_per_users_daily.csv")

In [180]:
def convert_str_to_date(date):
    if len(date) != 10:
        return datetime.datetime.strptime(str(date)[0:19], '%Y-%m-%d %H:%M:%S')
    else:
        return datetime.datetime.strptime(str(date)[0:10], '%Y-%m-%d')
    

In [181]:
# next_order['created_date'] = next_order['created_date'].apply(lambda x: convert_str_to_date(x))
last_30_orders_income['days'] = last_30_orders_income['days'].apply(lambda x: convert_str_to_date(x))

# Problem
   - What are the feature value of Individual customers?
   - In this case, We are calculating the Lifetime Value per each customer for the next 30 days, next month.

## Customer Life Time Value (Monthly)
   - Calculate the Next Purchase per User
       - when will the next purchase happen?
       - use the next frequency prediction per user.
   - Calculate the next order of amount per user related to frequency prediction
       - If there will be a purchase for any user, what would it be the amount of the purchase?

### Customer Lifetime Value Date Range
   - Next month will start the maximum date from the data set will finish by adding the maximum date to 30 days.

In [21]:
last_date = max(last_20_orders_per_users['created_date'])  
next_date = last_date + datetime.timedelta(days=30)

In [22]:
last_date, next_date

(Timestamp('2019-05-13 22:03:40'), Timestamp('2019-06-12 22:03:40'))

## 1 . the Next Purchase per User
   - Next purchase order prediction has been calculated on another script (next_order_prediction.ipynb)
   - It is calculated by Convolutional NN with 1 Dimensional Time Series Model. (Combination of Conv and LSTM layers)
   - It is calculated by the historical data of users.
   - In this case, in order to make sure to capture the right pattern by using Deep Learning, It is better to take at most 30 orders per user.
   - There are some issues when we are predicting the next purchase per user. First, some users of order history are not enough to predict future order frequency. Second, they still have churn probability related to their last recent order.

    ### ***a. Number of Order per User Problem***
       - On average, users have 1~2 orders. They don`t have 30 orders. It is better to assign them 0. 
       - By assigning 0 their prediction value will be more likely to close to 0. This also refers that the users of predicted values will not manage to occur inside the date range that we agreed.
       - More Orders will help us to figure out more 
       
    ### ***b. Churn Probability per User Problem***
        - The user who has higher recency, is more likely to become a churned user.
        - Users of maximum frequency values of distribution will show us what the ***average frequency*** value is.
        - How many orders on average would be completed per user when all users would have ***average frequency***.

### Frequency Distribution Of Users - All time

In [133]:
def calculate_time_diff(date, prev_date):
    date = datetime.datetime.strptime(str(date)[0:19], '%Y-%m-%d %H:%M:%S')
    prev_date = datetime.datetime.strptime(str(prev_date)[0:19], '%Y-%m-%d %H:%M:%S')
    return abs((date - prev_date).total_seconds()) / 60 / 60 / 24

In [134]:
last_30_orders_per_users['last_t_date'] = last_30_orders_per_users['last_t_date'].fillna(last_date)

In [139]:
last_30_orders_per_users_v2 = last_30_orders_per_users.query("last_t_date == last_t_date")
last_30_orders_per_users_v2['time_diff'] = last_30_orders_per_users_v2.apply(lambda row:
                                                                       calculate_time_diff(row['last_t_date'], row['created_date']), axis=1)
last_30_orders_per_users_v3 = last_30_orders_per_users_v2.groupby("user_id").agg({"time_diff": "mean"}).reset_index()
last_30_orders_per_users_v3.head()

,user_id,time_diff
0,user_0,16.061367
1,user_1,34.086801
2,user_10,15.484931
3,user_1000,48.947346
4,user_10000,133.089201


In [140]:
import plotly.express as px
fig = px.histogram(last_30_orders_per_users_v3, x="time_diff")
fig.show()

In [141]:
stats.mode([int(i) for i in last_30_orders_per_users_v3['time_diff']])

ModeResult(mode=array([12]), count=array([480]))

- This graph above clearly shows us that, each user has ***average 3 orders per user (2 + 1; additional 1 order refers to minimize the cost of making mistake) for predicting next month*** with the frequency of ***10 days (it is basically 12 days)***.

### Next order prediction calculation
 - Next order prediction calculates next 3 orders.
 - if 3rd predicted order or date still belongs to the date between last date and next month date, till the end of next month date it keeps predicting for each user.

### Frequency Distribution Of Users - Next Month

In [35]:
next_order = next_order.query("@last_date < created_date < @next_date")
next_order = next_order.sort_values(by=['user_id', 'created_date'], ascending=True)
next_order

,Unnamed: 0,created_date,user_id,time_diff,time_diff_norm,max_date,future_date
0,0,2019-05-15 08:12:00,user_10011,3.912289,0.060337,2019-05-13 07:46:46,2019-06-12 07:46:46
1,0,2019-05-25 02:56:00,user_10011,3.912289,0.060337,2019-05-13 07:46:46,2019-06-12 07:46:46
2,0,2019-06-03 21:40:00,user_10011,3.912289,0.060337,2019-05-13 07:46:46,2019-06-12 07:46:46
3,0,2019-05-15 08:14:00,user_10021,2.137358,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46
4,0,2019-05-20 16:28:00,user_10021,2.137358,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46
...,...,...,...,...,...,...,...
4067,0,2019-05-23 12:12:00,user_9977,6.650980,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46
4068,0,2019-06-09 03:15:00,user_9977,6.650980,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46
4069,0,2019-05-15 20:55:00,user_9980,9.948628,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46
4070,0,2019-06-09 17:50:00,user_9980,9.948628,0.060953,2019-05-13 07:46:46,2019-06-12 07:46:46


In [24]:
import plotly.express as px
fig = px.histogram(next_order, x="time_diff")
fig.show()

- This is related to each customer who will have next order during the day.
- I have wonder what the average order count per user is.

### Average Order Count per User

In [25]:
next_order.groupby("user_id").agg({"created_date": "count"}).reset_index()

,user_id,created_date
0,user_10011,3
1,user_10021,6
2,user_10026,2
3,user_1003,1
4,user_10032,4
...,...,...
1469,user_9935,1
1470,user_9966,2
1471,user_9977,2
1472,user_9980,2


In [26]:
import plotly.express as px
fig = px.histogram(next_order.groupby("user_id").agg({"created_date": "count"}).reset_index(), x="created_date")
fig.show()

- At least 1 purchase will be completed by each user.

### Churn Rate Approximation

In [125]:
print("number of user who are most like to pruchase next month :", len(next_order['user_id'].unique()))
print("number of user who have at least a purchase", len(last_30_orders_income['user_id'].unique()))
1 - (len(next_order['user_id'].unique()) / len(last_30_orders_income['user_id'].unique()))

number of user who are most like to pruchase next month : 1477
number of user who have at least a purchase 18766


0.9212938292656933

- 92% of users belong to the Churn Population. 
- This is the whole population of users.
- 1.4K users of 18.7K the whole population users are going to be engaged and will have purchase next month. 
- ***Monthly Churn Rate*** : ***Last Month Of Users that have purchase*** / ***(number of user will have purchase next month)***

In [121]:
last_month_start_date = next_date - datetime.timedelta(days=60)
last_month_order = last_30_orders_income.query("@last_month_start_date < days < @last_date")
last_month_order.head()

,user_id,transaction_value,transaction_count,days
795809,user_0,44.380833,12,2019-04-14
795810,user_10000,31.270000,4,2019-04-14
795811,user_10004,2.550000,1,2019-04-14
795812,user_10014,26.798000,5,2019-04-14
795813,user_10018,13.616667,3,2019-04-14


In [122]:
monthly_churn_rate = 1 - (len(next_order['user_id'].unique()) / len(last_month_order['user_id'].unique()))
monthly_churn_rate

0.8694768469423825

Monthly Churn Rate is much higher than overal churn rate. 

## 2. The Next order of Amount per User
 - The Next order of Amount per User has been calculated on other script (customer_next_transaction_value_1d_cnn_time_series_implementation.ipynb)
 - This metric is also been calculated by Recurrent-LSTM NN.
 - User of transactions is sequentially gathered. Only the last 30 purchases per user have been included in to feature set.

In [170]:
user_12622 = last_30_orders_income.query("user_id == 'user_12622'"
                                        ).sort_values(by='days').reset_index(drop=True).reset_index()[['index', 'transaction_value']]
max_order = max(user_12622['index'])
user_12622_predicted = weekly_income.query("user_id == 'user_12622'").reset_index(drop=True).reset_index()[['index', 'prediction_values']]
user_12622_predicted['index'] = user_12622_predicted['index'] + 1 + max_order

In [174]:
vis = pd.concat([user_12622_predicted, user_12622.rename(columns={"transaction_value": "prediction_values"})]).sort_values(by='index')

fig = px.line(vis.rename(columns={"prediction_values": "purchase payment amount", "index": "order number"}), 
              x='order number', y="purchase payment amount", title='Purchase Payment Amount Per Order And Future Predicted Values')
fig.show()

## 3. Customer Lifetime Value

In [190]:
sum(weekly_income['prediction_values'])

1014070871.3884847

In [193]:
counter = 12
total_value = [{"prev_month": 13, "total_income": sum(weekly_income['prediction_values'])}]
for i in range(0, 12):
    _start, _end = (i+1) * 30, i * 30 
    _start_date, _end_date = last_date - datetime.timedelta(days = _start), last_date - datetime.timedelta(days = _end)
    _income = last_30_orders_income.query("@_start_date < days < @_end_date")
    # _income['values'] = _income['transaction_value'] * _income['transaction_count']
    print(_start_date, _end_date)
    print(sum(_income['transaction_value']))
    total_value.append({"prev_month": counter, "total_income": sum(_income['transaction_value'])})
    counter -= 1

2019-04-13 22:03:40 2019-05-13 22:03:40
1887043349.7168534
2019-03-14 22:03:40 2019-04-13 22:03:40
114194161192.16664
2019-02-12 22:03:40 2019-03-14 22:03:40
10467140617.732702
2019-01-13 22:03:40 2019-02-12 22:03:40
1356065420.055386
2018-12-14 22:03:40 2019-01-13 22:03:40
38831049956.50129
2018-11-14 22:03:40 2018-12-14 22:03:40
37841236096.33124
2018-10-15 22:03:40 2018-11-14 22:03:40
26146787488.03553
2018-09-15 22:03:40 2018-10-15 22:03:40
958733443.5284504
2018-08-16 22:03:40 2018-09-15 22:03:40
330850757.18823856
2018-07-17 22:03:40 2018-08-16 22:03:40
1471103586.6650004
2018-06-17 22:03:40 2018-07-17 22:03:40
2490087.5101497523
2018-05-18 22:03:40 2018-06-17 22:03:40
2228570.558786139


In [196]:
fig = px.line(pd.DataFrame(total_value), 
              x='prev_month', y="total_income", title='Month Of Revenues (13th Month is CLV Predicted Value)')
fig.show()

In [267]:
comparison_per_user = pd.merge(last_30_orders_income.groupby("user_id").agg({"transaction_value": "mean"}).reset_index(), 
                               weekly_income.groupby("user_id").agg({"prediction_values": "median"}), on='user_id', how='inner')

In [268]:
comparison_per_user['diff'] = comparison_per_user['transaction_value'] - comparison_per_user['prediction_values']

In [269]:
q1, q3 = np.quantile(comparison_per_user['diff'], 0.1), np.quantile(comparison_per_user['diff'], 0.9)

In [270]:
def decision_of_significany(diff, q1, q3):
    decision = 'no_change'
    if diff < q1:
        decision = 'significant_decrease'
    if diff > q3:
        decision = 'significant_decrease'
    return decision


comparison_per_user['significany'] = comparison_per_user['diff'].apply(lambda x: decision_of_significany(x, q1, q3))

In [271]:
vis = comparison_per_user.groupby("significany").agg({"user_id": "count"}).reset_index()
vis

,significany,user_id
0,no_change,1181
1,significant_decrease,296


In [272]:
fig = go.Figure(data=[
    go.Bar(name='LA Zoo', x=vis['significany'], y=vis['user_id'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [286]:
_df = last_30_orders_income.query("user_id == 'user_4955'").sort_values(by='days').reset_index(drop=True).reset_index()[['index', 'transaction_value']]
max_order = max(_df['index'])
_df2 = weekly_income.query("user_id == 'user_4955'").reset_index(drop=True).reset_index()
_df2 = _df2[['index', 'prediction_values']].rename(columns={"prediction_values": "transaction_value"})
_df2['index'] = _df2['index'] + 1 + max_order

In [287]:
vis = pd.concat([_df, _df2])
fig = px.line(vis, 
              x='index', y="transaction_value", title='Month Of Revenues (13th Month is CLV Predicted Value)')
fig.show()